# Test Coriolis Term of Shallow Water Physics on a mesh
A simple gaussian test case is run to see the effect of the coriolis term

In [19]:
include("mode_forward/time_steppers.jl")
include("mode_init/MPAS_Ocean.jl")
include("visualization.jl")

using PyPlot
using PyCall

patch      = pyimport("matplotlib.patches")
col        = pyimport("matplotlib.collections")
animation  = pyimport("matplotlib.animation")
ipydisplay = pyimport("IPython.display")

PyObject <module 'IPython.display' from '/home/rrs/anaconda3/envs/MPAS_Ocean/lib/python3.8/site-packages/IPython/display.py'>

## load mesh from a file

In [3]:
mpasOcean = MPAS_Ocean(mesh_directory = "MPAS_O_Shallow_Water/Mesh+Initial_Condition+Registry_Files/Periodic",
                    base_mesh_file_name = "base_mesh.nc",
                    mesh_file_name = "mesh.nc")
mpasOcean.myNamelist

Namelist(1000.0, "uniform", #undef, false, false, true, "Periodic", "Forward_Backward", #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, 0.0, 0.0001, 2.0000000000000002e-11, 10.0, 1000.0, 0.001, "default", "centered", false, 100.0, 1.0e6, 2.2360679774997896e6, 180.0, 0.0, 500.0, 500.0, 1.0, 0.0, 0.5, 12500.0)

## define dt, duration of the simulation, number of snapshots of the sea surface height to record, and arrays for recording it

In [4]:
# calculate dt based on CFL condition
dt = 0.1 * minimum(mpasOcean.dcEdge) / sqrt( gravity  * maximum(mpasOcean.bottomDepth) )

nFrames = 100

simTimeBetweenFrames = 3*dt

"dt: ", dt

("dt: ", 1.0101525445522077)

# Run the simulation

## gaussian initial condition function

In [5]:
function gaussianInit!(mpasOcean)
    # define gaussian initial condition
    for iCell in 1:mpasOcean.nCells
        mpasOcean.sshCurrent[iCell] = exp(
            (
                -(mpasOcean.yCell[iCell] - 20000)^2
                -(mpasOcean.xCell[iCell] - 30000)^2
            ) / ( 1.0e7 )
        )
    end

    mpasOcean.normalVelocityCurrent = zeros(mpasOcean.nEdges)
end

gaussianInit! (generic function with 1 method)

## integrate

In [6]:
# solve it, integrate!
## forward euler
gaussianInit!(mpasOcean)

sshOverTimeCori = zeros(Float64, (nFrames, mpasOcean.nCells))

sshOverTimeCori[1,:] = mpasOcean.sshCurrent[:]

for frame in 2:nFrames
    forwardBackward!(mpasOcean, dt, simTimeBetweenFrames)
    sshOverTimeCori[frame,:] = mpasOcean.sshCurrent[:]
end


## animate solution

In [27]:
fig = figure()

cMax = maximum(abs.(sshOverTimeCori))

_, ax, _, collection = heatMapMesh(mpasOcean, sshOverTimeCori[1,:], fig=fig, cMin=-cMax, cMax=cMax)

function nextFrame(j)
    i = j + 1
    
    collection.set_array(sshOverTimeCori[i,:])
    
    return ax
end

anim = animation.FuncAnimation(fig, nextFrame, frames=nFrames)

PyObject <matplotlib.animation.FuncAnimation object at 0x7f09fd5cfa90>

In [28]:
ipydisplay.HTML(anim.to_html5_video())

PyObject <IPython.core.display.HTML object>

In [29]:
# anim.save("plots/test1/forwardeuler_ssh_test1.mp4")
# ipydisplay.Video("plots/test1/forwardeuler_ssh_test1.mp4")

## reset simulation and run without coriolis force

In [30]:
# reset
gaussianInit!(mpasOcean)

# zero out coriolis coeffecient
mpasOcean.fEdge = zeros(Float64, mpasOcean.nEdges)

sshOverTimeNoCori = zeros(Float64, (nFrames, mpasOcean.nCells))

for frame in 1:nFrames
    forwardBackward!(mpasOcean, dt, simTimeBetweenFrames)
    sshOverTimeNoCori[frame,:] = mpasOcean.sshCurrent[:]
end

# Compare with and without coriolis term side by side

In [31]:
fig = figure()

ax1 = fig.add_subplot(1, 3, 1)
ax2 = fig.add_subplot(1, 3, 2)
ax3 = fig.add_subplot(1, 3, 3)

cMax1 = maximum(abs.(sshOverTimeCori))
cMax2 = maximum(abs.(sshOverTimeNoCori))
cMax3 = maximum(abs.(sshOverTimeCori .- sshOverTimeNoCori))


_, _, _, col1 = heatMapMesh(mpasOcean, sshOverTimeCori[1,:]; fig=fig, ax=ax1, cMin=-cMax1, cMax=cMax1)
ax1.set_title("With Coriolis")
ax1.axis("off")

_, _, _, col2 = heatMapMesh(mpasOcean, sshOverTimeNoCori[1,:]; fig=fig, ax=ax2, cMin=-cMax2, cMax=cMax2)
ax2.set_title("Without Coriolis")
ax2.axis("off")

_, _, _, col3 = heatMapMesh(mpasOcean, sshOverTimeCori[1,:] .- sshOverTimeNoCori[1,:]; fig=fig, ax=ax3, cMin=-cMax3, cMax=cMax3)
ax3.set_title("Difference")
ax3.axis("off")


tight_layout()

function nextFrame(j)

    i = j+1

    col1.set_array(sshOverTimeCori[i,:])
    col2.set_array(sshOverTimeNoCori[i,:])
    col3.set_array(sshOverTimeCori[i,:] - sshOverTimeNoCori[i,:])

    fig.suptitle("frame $j of $nFrames")

    return [ax1, ax2, ax3]

end

anim = animation.FuncAnimation(fig, nextFrame, frames=nFrames)

PyObject <matplotlib.animation.FuncAnimation object at 0x7f09fa7bf100>

In [32]:
ipydisplay.HTML(anim.to_html5_video())

PyObject <IPython.core.display.HTML object>

In [33]:
# anim.save("plots/test2/forwardeuler_vs_forwardbackward.mp4")
# ipydisplay.Video("plots/test2/forwardeuler_vs_forwardbackward.mp4")